# Data Modelling and Evaluation

## Objectives

* Fulfil project objective - use the data insights gained from the DataVisualisation notebook to train the ML model to make predictions on whether leaves are infected or not

## Inputs

* inputs/cherry-leaves/cherry-leaves/train
* inputs/cherry-leaves/cherry-leaves/test
* inputs/cherry-leaves/cherry-leaves/validation
* image shape embeddings 

## Outputs

* 

## Additional Comments

* No additional comments




---

# Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import tensorflow as tf
sns.set_style("white")
from matplotlib.image import imread
from PIL import Image 

---

# Change working directory

Access the current directory with os.getcwd()

In [ ]:
cwd = os.getcwd()

Set new directory

In [ ]:
os.chdir('/workspace/Mildew-Detection')
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Set Input Directories


In [ ]:
my_data_dir = '/workspace/Mildew-Detection/inputs/cherry-leaves/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

# Set Output Directory

In [ ]:
# If new version needed, just change version variable
version = 'v1'

file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

---

# Set Label Names

In [ ]:
labels = os.listdir(train_path)
print('Label for the images are', labels)

---

---

# Conclusions and Next Steps

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.